In [36]:
import time
from terraaws.utilities import S3
from terraaws.utilities import Network
from terraaws.network import VPC
from terraaws.compute import EC2
from terraaws.database import RDS

In [37]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
#project s3 bucket
s3 = S3(profile_name = 'mjn')
prj_bucket = s3.create_private_bucket('tripoli-env')

In [39]:
#project vpc, security group, route table, 2 subnets for database, public subnet
vpc = VPC(profile_name = 'mjn')
prj_vpc, prj_sg, prj_rt = vpc.create_base_vpc('213.213.48.0/24', 'tripoli')
subnets = vpc.create_subnet('213.213.48.32/27', 'us-west-2b', 'tripoli-dbsubnet-01')
subnets = vpc.create_subnet('213.213.48.64/27', 'us-west-2c', 'tripoli-dbsubnet-02')
subnets = vpc.create_subnet('213.213.48.96/27', 'us-west-2a', 'tripoli-publicsubnet', public = True)
intgateway = vpc.create_internet_gateway('tripoli-ig')
int_route = vpc.create_internet_route('0.0.0.0/0')
publicsubnet_id = Network.get_subnet_id(subnets, 'tripoli-publicsubnet')
securitygroup_id = Network.get_securitygroup_id(prj_sg, 'tripoli-base-securitygroup')

In [40]:
#amazon linux ec2
ec2 = EC2(profile_name = 'mjn')
prj_ec2 = ec2.launch_amazon_linux2('tripoli-ec2', publicsubnet_id, [securitygroup_id], 'tripoli_key')

In [43]:
#database subnet group, amazon mysql rds
rds = RDS(profile_name = 'mjn')
subnet_ids_db = [Network.get_subnet_id(subnets, 'tripoli-dbsubnet-01'), 
                 Network.get_subnet_id(subnets, 'tripoli-dbsubnet-02')]
dbsubnetgrp = rds.create_db_subnet_group('tripolidb', subnet_ids_db)
prj_db = rds.launch_mysql_rds('*****',
                              '*****',
                              '*****',
                              'tripolidb-subnetgroup',
                              securitygroup_id)

In [ ]:
#write configs
put_status = s3.put_object(prj_bucket, 'tripoli-env', 'tripoli-s3-config-' + str(int(time.time())) + '.json')
put_status = s3.put_object(prj_vpc, 'tripoli-env', 'tripoli-vpc-config-' + str(int(time.time())) + '.json')
put_status = s3.put_object(prj_sg, 'tripoli-env', 'tripoli-sg-config-' + str(int(time.time())) + '.json')
put_status = s3.put_object(subnets, 'tripoli-env', 'tripoli-subnet-config-' + str(int(time.time())) + '.json')
put_status = s3.put_object(intgateway, 'tripoli-env', 'tripoli-intgateway-config-' + str(int(time.time())) + '.json')
put_status = s3.put_object(int_route, 'tripoli-env', 'tripoli-route-table-config-' + str(int(time.time())) + '.json')
put_status = s3.put_object(prj_ec2, 'tripoli-env', 'tripoli-ec2-config-' + str(int(time.time())) + '.json')
put_status = s3.put_object(dbsubnetgrp, 'tripoli-env', 'tripoli-dbsubnet-config-' + str(int(time.time())) + '.json')
put_status = s3.put_object(prj_db, 'tripoli-env', 'tripoli-db-config-' + str(int(time.time())) + '.json')